<a href="https://colab.research.google.com/github/Guzel-Gizatullina/heart_failer_predicton/blob/main/%D0%94%D0%97_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import numpy as np


In [60]:
sales_data= pd.read_csv('/content/monthly-sales-of-company-x-jan-6.csv')

In [61]:
sales_data.head(3)

,Month,Count
0,1965-01,154
1,1965-02,96
2,1965-03,73


In [62]:
car_sales_data = pd.read_csv('/content/monthly-car-sales-in-quebec-1960.csv')
car_sales_data.head(3)

,Month,Count
0,1960-01,6550
1,1960-02,8728
2,1960-03,12026


In [63]:
airline_passengers_data = pd.read_csv('/content/international-airline-passengers.csv')
airline_passengers_data.head(3)

,Month,International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60
0,1949-01,112.0
1,1949-02,118.0
2,1949-03,132.0


In [64]:
# Просмотр заголовков для уточнения структуры данных
{
    "sales_data_columns": sales_data.columns.tolist(),
    "car_sales_data_columns": car_sales_data.columns.tolist(),
    "airline_passengers_data_columns": airline_passengers_data.columns.tolist()
}


{'sales_data_columns': ['Month', 'Count'],
 'car_sales_data_columns': ['Month', 'Count'],
 'airline_passengers_data_columns': ['Month',
  'International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60']}

In [65]:
from statsmodels.tsa.stattools import adfuller

# Функция для выполнения теста Дики-Фуллера
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    return dftest[1]  # Возвращаем p-value

# Подготовка данных
sales_data['Month'] = pd.to_datetime(sales_data['Month'])
car_sales_data['Month'] = pd.to_datetime(car_sales_data['Month'])

# Установка индекса времени
sales_data.set_index('Month', inplace=True)
car_sales_data.set_index('Month', inplace=True)


# Проверка стационарности
sales_p_value = test_stationarity(sales_data['Count'])
car_sales_p_value = test_stationarity(car_sales_data['Count'])

sales_p_value, car_sales_p_value

(0.9888894205170052, 0.6632691049832861)

In [67]:
# Переименование столбца для данных о пассажирах авиалиний для упрощения
airline_passengers_data.rename(columns={airline_passengers_data.columns[1]: 'Count'}, inplace=True)

# Преобразование столбца 'Month' в формат datetime
airline_passengers_data.index = pd.to_datetime(airline_passengers_data.index)

# Проверка стационарности после корректировки
airline_passengers_p_value = test_stationarity(airline_passengers_data['Count'])

sales_p_value, car_sales_p_value, airline_passengers_p_value


(0.9888894205170052, 0.6632691049832861, nan)

In [68]:
# Исправление для корректной обработки индекса и столбца данных
airline_passengers_data = airline_passengers_data.dropna()  # Удаление NaN значений

# Повторная проверка стационарности для данных о пассажирах авиалиний
airline_passengers_p_value = test_stationarity(airline_passengers_data['Count'])
airline_passengers_p_value


0.991880243437641

# Eсть три нестационарных временных ряда. Чтобы преобразовать их в стационарные, мы можем применить дифференцирование; применим первое разностное преобразование к каждому из рядов и затем проверим их на стационарность снова. ​

In [69]:
# Применение первого разностного преобразования к каждому ряду
sales_diff = sales_data['Count'].diff().dropna()
car_sales_diff = car_sales_data['Count'].diff().dropna()
airline_passengers_diff = airline_passengers_data['Count'].diff().dropna()

# Проверка стационарности после дифференцирования
sales_diff_p_value = test_stationarity(sales_diff)
car_sales_diff_p_value = test_stationarity(car_sales_diff)
airline_passengers_diff_p_value = test_stationarity(airline_passengers_diff)

sales_diff_p_value, car_sales_diff_p_value, airline_passengers_diff_p_value


(0.0955092507855888, 5.8447525992768446e-08, 0.05421329028382478)

#Резюмирование проделонного анализа.

После дифференцирования результаты теста Дики-Фуллера следующие:

Для продаж компании X p-value уменьшилось до 0.096, что является значительным улучшением, но все еще выше критического уровня 0.05 для строгой стационарности.
Для продаж автомобилей в Квебеке p-value равно приблизительно
5.84
×
1
0
−
8
5.84×10
−8
 , что значительно ниже 0.05, указывая на стационарность временного ряда после дифференцирования.
Для данных о пассажирах авиалиний p-value составляет 0.054, что находится очень близко к 0.05, поэтому можно рассматривать ряд как почти стационарный, хотя для более строгой стационарности можно применить дополнительные преобразования.
На основе этих результатов, продажи автомобилей в Квебеке теперь стационарны, в то время как для двух других рядов может потребоваться дополнительное дифференцирование или другие методы преобразования для достижения полной стационарности.